# AI-Powered A/B Testing and Budget Allocation for Marketing Campaigns

**Copyright (c) 2026 Shrikara Kaudambady. All rights reserved.**

This notebook simulates an AI Marketing Agent that uses a **Multi-Armed Bandit (MAB)** algorithm to autonomously optimize a marketing campaign. Instead of a traditional A/B test where budget is split evenly, the MAB agent dynamically allocates budget in real-time to the best-performing ad variants, maximizing conversions and minimizing wasted spend.

### 1. Setup and Library Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import beta

sns.set_theme(style="whitegrid")

### 2. The Marketing Campaign Environment
We define our ad variants (the 'arms' of the bandit). Each arm is a combination of ad copy and a channel. We assign each a **secret, true conversion rate** that the AI agent's goal is to discover.

In [ ]:
ad_variants = {
    'Copy A on Facebook': 0.05,  # 5% true conversion rate
    'Copy B on Facebook': 0.04,  # 4% true conversion rate
    'Copy A on Google':   0.08,  # 8% true conversion rate (The best arm)
    'Copy B on Google':   0.065, # 6.5% true conversion rate
    'Copy C on Instagram':0.03   # 3% true conversion rate
}

class MarketingEnvironment:
    def __init__(self, variants):
        self.variants = variants
    
    def pull_arm(self, arm_name):
        """Simulates showing an ad once. Returns 1 for a conversion, 0 for no conversion."""
        conversion_rate = self.variants.get(arm_name, 0)
        if np.random.rand() < conversion_rate:
            return 1 # Success (conversion)
        else:
            return 0 # Failure (no conversion)

env = MarketingEnvironment(ad_variants)
print("Marketing environment created with 5 ad variants.")

### 3. The AI Agent: Thompson Sampling
This agent uses a Bayesian approach. For each arm, it maintains a Beta distribution representing the probable range of the true conversion rate. To choose an arm, it samples a random value from each distribution and picks the highest. This elegantly balances exploring new options and exploiting known good ones.

In [ ]:
class ThompsonSamplingAgent:
    def __init__(self, arms):
        self.arms = arms
        # Initialize alpha and beta for the Beta distribution. Start with 1 for each.
        self.results = {arm: {'alpha': 1, 'beta': 1} for arm in self.arms}
        
    def select_arm(self):
        """Selects an arm by sampling from each arm's Beta distribution."""
        max_sample = -1
        best_arm = None
        
        for arm in self.arms:
            # Draw a random sample from the Beta(alpha, beta) distribution
            dist = self.results[arm]
            sample = np.random.beta(dist['alpha'], dist['beta'])
            
            if sample > max_sample:
                max_sample = sample
                best_arm = arm
        
        return best_arm

    def update(self, arm_name, reward):
        """Updates the distribution for the chosen arm based on the result."""
        if reward == 1:
            self.results[arm_name]['alpha'] += 1 # Increment successes
        else:
            self.results[arm_name]['beta'] += 1 # Increment failures

### 4. Running the A/B Test Simulation
We'll run the simulation for a set number of impressions and log the agent's choices and the outcomes.

In [ ]:
n_impressions = 20000
agent = ThompsonSamplingAgent(list(ad_variants.keys()))
log = []

for i in range(n_impressions):
    chosen_arm = agent.select_arm()
    reward = env.pull_arm(chosen_arm)
    agent.update(chosen_arm, reward)
    log.append({'impression': i, 'chosen_arm': chosen_arm, 'reward': reward})

log_df = pd.DataFrame(log)
print("Simulation complete.")

### 5. Analysis and Visualization
Let's analyze the results to see how the agent performed.

In [ ]:
print("--- Final Campaign Results ---")
summary = log_df.groupby('chosen_arm')['reward'].agg(['sum', 'count']).rename(columns={'sum':'conversions'})
summary['conversion_rate'] = summary['conversions'] / summary['count']
summary['true_rate'] = summary.index.map(ad_variants)
print(summary.sort_values(by='conversion_rate', ascending=False))

best_arm_name = max(ad_variants, key=ad_variants.get)
print(f"\nThe agent correctly identified '{best_arm_name}' as the best variant and allocated most of its budget (impressions) to it.")

# Plot the evolution of choices
plt.figure(figsize=(15, 7))
sns.scatterplot(x='impression', y='chosen_arm', data=log_df, hue='chosen_arm', alpha=0.1)
plt.title('Agent Arm Selection Over Time')
plt.xlabel('Impression Number')
plt.ylabel('Ad Variant')
plt.legend(title='Ad Variant', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()
print("The plot above shows how the agent explores all options at the beginning but quickly starts to exploit the best-performing arm ('Copy A on Google').")

#### Visualizing the Agent's "Beliefs"

This plot shows the final Beta probability distributions for each arm. The taller and narrower the distribution, the more confident the agent is about the true conversion rate. The distribution for the best arm is clearly shifted to the right.

In [ ]:
plt.figure(figsize=(15, 8))
x = np.linspace(0, 0.15, 200)
for arm in agent.arms:
    dist = agent.results[arm]
    y = beta.pdf(x, dist['alpha'], dist['beta'])
    plt.plot(x, y, label=f"{arm} (α={dist['alpha']}, β={dist['beta']})")

plt.title("Agent's Final Beliefs (Beta Distributions) for Each Ad Variant's Conversion Rate", fontsize=14)
plt.xlabel("Conversion Rate")
plt.ylabel("Probability Density")
plt.legend()
plt.show()